In [4]:
import h5py
with h5py.File('D:\SLEAP 172\FixationSideCam/1201_prediction_analysis.h5', 'r') as f:
    occupancy_matrix = f['track_occupancy'][:]
    tracks_matrix = f['tracks'][:]

print(occupancy_matrix.shape)
print(tracks_matrix.shape)

(293, 1)
(1, 2, 3, 293)


In [5]:
with h5py.File( 'D:\SLEAP 172\FixationSideCam/1201_prediction_analysis.h5', 'r') as f:
    dset_names = list(f.keys())
    locations = f["tracks"][:].T
    node_names = [n.decode() for n in f["node_names"][:]]

print("===filename===")
print('/content/1201_prediction_analysis.h5')
print()

print("===HDF5 datasets===")
print(dset_names)
print()

print("===locations data shape===")
print(locations.shape)
print()

print("===nodes===")
for i, name in enumerate(node_names):
    print(f"{i}: {name}")
print()

===filename===
/content/1201_prediction_analysis.h5

===HDF5 datasets===
['edge_inds', 'edge_names', 'instance_scores', 'labels_path', 'node_names', 'point_scores', 'provenance', 'track_names', 'track_occupancy', 'tracking_scores', 'tracks', 'video_ind', 'video_path']

===locations data shape===
(293, 3, 2, 1)

===nodes===
0: snout
1: eye
2: ear



In [6]:
frame_count, node_count, _, instance_count = locations.shape

print("frame count:", frame_count)
print("node count:", node_count)
print("instance count:", instance_count)

frame count: 293
node count: 3
instance count: 1


In [10]:
import sleap

# This prevents TensorFlow from allocating all the GPU memory, which leads to issues on
# some GPUs/platforms:
sleap.disable_preallocation()

# This would hide GPUs from the TensorFlow altogether:
# sleap.use_cpu_only()

# Print some info:
sleap.versions()
sleap.system_summary()

SLEAP: 1.3.4
TensorFlow: 2.7.0
Numpy: 1.21.6
Python: 3.7.12
OS: Windows-10-10.0.16299-SP0
GPUs: 2/2 available
  Device: /physical_device:GPU:0
         Available: True
        Initalized: False
     Memory growth: True
  Device: /physical_device:GPU:1
         Available: True
        Initalized: False
     Memory growth: True


In [14]:
predictor = sleap.load_model("D:\SLEAP 172\FixationSideCam\models/241125_012328.single_instance.zip")

In [15]:
import numpy as np
def get_predictions(filename):

  video=sleap.load_video(filename)
  total_frames=video.num_frames
  labels = predictor.predict(video)

  return labels, video

labels, video =get_predictions("D:\SLEAP 172\FixationSideCam\Videos/EC01_results_2024-11-08T10_20_2_Trial_23.avi")

Predicting... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% ETA: 0:00:00 96.9 FPS

In [106]:
import numpy as np

total_frames=video.num_frames

pts = labels.numpy()


for i in range (total_frames):
    A=np.matrix(pts[i])
    B= A[:,0]
    C= A[:,1]
    x_predictions=B.transpose()
    y_predictions=C.transpose()
    
    def get_fixation_predictions(x_predictions,y_predictions,fix_start_frame, fix_end_frame):
        x_start=x_predictions[fix_start_frame]
        x_end=x_predictions[fix_end_frame]
        y_start=y_predictions[fix_start_frame]
        y_end=y_predictions[fix_end_frame]
    
        fixation_predictions=x_start,y_start,x_end,y_end
        print(fixation_predictions)
    
    

get_fixation_predictions(x_predictions,y_predictions,275,275)


IndexError: index 275 is out of bounds for axis 0 with size 1

In [32]:
import pandas as pd
Time= pd.read_csv("D:\SLEAP 172\FixationSideCam/EC01.csv")

In [33]:
def get_fixation_period(session_name):
    filtered_data = Time[Time['sess'] == session_name]
    fixation_period_table = pd.DataFrame(filtered_data[['TrialNum', 'FixationStarted_time', 'FixationCompleted_time']])

    return fixation_period_table
get_fixation_period('2024-11-08T10_20_28')

,TrialNum,FixationStarted_time,FixationCompleted_time
18466,1,10:22:50.806412,10:22:50.847065
18467,2,10:23:01.846259,10:23:01.880627
18468,3,10:23:14.284211,10:23:14.317158
18469,4,10:23:33.454464,10:23:33.501145
18470,5,10:23:42.415872,10:23:42.465216
...,...,...,...
18682,217,11:44:18.914700,11:44:23.539084
18683,218,11:45:21.072614,11:45:25.695974
18684,219,11:48:00.458956,11:48:05.482624
18685,220,11:52:46.461273,11:52:51.484697


In [34]:
import pandas as pd
Frame=pd.read_csv('D:\SLEAP 172\FixationSideCam/FixationSideCam_stamps 1108.csv',delimiter=';')

In [35]:
fixation_period_table = get_fixation_period('2024-11-08T10_20_28')

In [38]:
def get_fixation_frame(i,fixation_period_table):
  fixation_period_trial_i=fixation_period_table[fixation_period_table['TrialNum']==i]
  fix_start_time=fixation_period_trial_i['FixationStarted_time'].iloc[0]
  fix_end_time=fixation_period_trial_i['FixationCompleted_time'].iloc[0]

  Frame['FrameTime'] = pd.to_datetime(Frame['FrameTime'])
  fix_period = Frame.loc[(Frame['TrialNum'] == i) & (Frame['FrameTime'].between(fix_start_time, fix_end_time)), 'FrameNum']
  fix_start_frame=fix_period.iloc[0]
  fix_end_frame=fix_period.iloc[-1]


  print(fix_start_frame, fix_end_frame)

get_fixation_frame(23,fixation_period_table)

275 275


In [46]:
def get_fixation_predictions(x_predictions,y_predictions,fix_start_frame, fix_end_frame):
    
    x_start=x_predictions[fix_start_frame]
    x_end=x_predictions[fix_end_frame]
    y_start=y_predictions[fix_start_frame]
    y_end=y_predictions[fix_end_frame]
    
    fixation_predictions=x_start,y_start,x_end,y_end
    print(fixation_predictions)
    
    

get_fixation_predictions(x_predictions,y_predictions,275,275)

IndexError: index 275 is out of bounds for axis 0 with size 1